In [62]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split

In [2]:
data = pd.read_csv("last_train.csv")
data_label = pd.read_csv("last_label.csv")

In [57]:
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.kernel_ridge import KernelRidge

In [67]:
train = data.iloc[:-100,:]
test = data.iloc[-100:,:]
train_label = data_label.iloc[:-100,:]
test_label = data_label.iloc[-100:,:]
train_array = np.array(train)
train_label_array = np.array(train_label)

In [77]:
train_label_array.reshape(-1,1).shape

(269, 1)

In [58]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.09,
                             learning_rate=0.009, 
                              n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213,
                             random_state =7, nthread = -1)
GBoost = GradientBoostingRegressor(max_depth=2, learning_rate=0.005,min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

KRR = KernelRidge(alpha=0.1, kernel='polynomial', degree=1, coef0=5)

ENet = ElasticNet(alpha=0.002, l1_ratio=.6, random_state=3)

lasso = Lasso(alpha =0.002, random_state=1)

RForest = RandomForestRegressor(n_estimators=100,max_depth=2,min_samples_leaf=2\
                               ,min_weight_fraction_leaf=0.5,max_leaf_nodes=2\
                                ,random_state=6)

In [85]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        
        for i, model in enumerate(self.base_models):
            
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                
                self.base_models_[i].append(instance)
                
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
      
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)


In [86]:
stacked_averaged_models = StackingAveragedModels(base_models = (ENet,KRR,lasso),meta_model =  GBoost)

In [87]:
stacked_averaged_models.fit(train_array,train_label_array)
#res = stacked_averaged_models.predict(test)

ValueError: shape mismatch: value array of shape (54,1) could not be broadcast to indexing result of shape (54,)

In [31]:
result = np.expm1(res)
real_value = np.expm1(data_label)

In [32]:
result = pd.DataFrame(result)

In [34]:
result.head()

,0
0,12441.605130
1,12225.087726
2,12249.160633
3,12447.420543
4,12263.207055


In [45]:
x_range = [x for x in range(1,len(real_value)+1)]

In [47]:
x2_range = [x for x in range(len(real_value)-100, len(real_value))]

In [88]:
%matplotlib tk
plt.figure(1)
plt.plot(x_range,real_value)
plt.plot(x2_range,result)